In [1]:
!pip install open3d

   ---------------------------------------- 0.0/62.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/62.9 MB 2.0 MB/s eta 0:00:31
   ---------------------------------------- 0.1/62.9 MB 656.4 kB/s eta 0:01:36
   ---------------------------------------- 0.1/62.9 MB 880.9 kB/s eta 0:01:12
   ---------------------------------------- 0.2/62.9 MB 919.0 kB/s eta 0:01:09
   ---------------------------------------- 0.2/62.9 MB 1.2 MB/s eta 0:00:53
   ---------------------------------------- 0.3/62.9 MB 1.2 MB/s eta 0:00:52
   ---------------------------------------- 0.4/62.9 MB 1.5 MB/s eta 0:00:42
   ---------------------------------------- 0.5/62.9 MB 1.6 MB/s eta 0:00:39
   ---------------------------------------- 0.7/62.9 MB 1.9 MB/s eta 0:00:34
    --------------------------------------- 0.8/62.9 MB 2.0 MB/s eta 0:00:31
    --------------------------------------- 0.9/62.9 MB 2.0 MB/s eta 0:00:31
    --------------------------------------- 1.1/62.9 MB 2.2 MB/s eta 0:00:29



[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [48]:
import open3d as o3d
import numpy as np
import os
import sys
sys.path.append('..')

In [3]:
file_path = 'fragment.ply'
pcd = o3d.io.read_point_cloud(file_path)
print(pcd)

PointCloud with 196133 points.


In [4]:
print(np.asarray(pcd.points))

[[0.65234375 0.84686458 2.37890625]
 [0.65234375 0.83984375 2.38430572]
 [0.66737998 0.83984375 2.37890625]
 ...
 [2.00839925 2.39453125 1.88671875]
 [2.00390625 2.39488506 1.88671875]
 [2.00390625 2.39453125 1.88793314]]


In [5]:
o3d.visualization.draw_geometries([pcd],zoom=0.3412, front = [0.4257, -0.2125, -0.8795],
                                 lookat=[2.6172, 2.0475, 1.532],
                                 up=[-0.0694, -0.9768, 0.2024])

[Open3D WARNING] [ViewControl] SetViewPoint() failed because window height and width are not set.


In [6]:
o3d.visualization.draw_geometries([pcd],width=1920, height=1080, left=50, top=50)

# Voxel DownSampling


In [15]:
downpcd=pcd.voxel_down_sample(voxel_size=0.05)
o3d.visualization.draw_geometries([downpcd],zoom=0.3412, front = [0.4257, -0.2125, -0.8795],
                                 lookat=[2.6172, 2.0475, 1.532],
                                 up=[-0.0694, -0.9768, 0.2024])

# Vertex Normal Estimation


In [16]:
downpcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))
o3d.visualization.draw_geometries([pcd],zoom=0.3412, front = [0.4257, -0.2125, -0.8795],
                                 lookat=[2.6172, 2.0475, 1.532],
                                 up=[-0.0694, -0.9768, 0.2024],
                                 point_show_normal=True)

'estimate_normals' --> computes the normal for every point. The function finds adjacent points and calculations the principal axis of the adjacent points using covariance analysis

The function takes an instance of 'KDTreeSearchParamHybrid' class as an argument. KDTree search parameters for hybrid KNN and radius search.
property 'max_nn'
At maximum, max_nn neighbors will be searched.

property 'radius'
Search radius.

# Access estimated vertex normal
Estimated normal vectors can be retrived from the normals variable of downpcd

In [19]:
print(downpcd.normals[0])

[ 0.85641574  0.01693013 -0.51600915]


In [20]:
#the normal vectors of first 10 points
print(np.asarray(downpcd.normals)[:10,:])

[[ 8.56415744e-01  1.69301342e-02 -5.16009150e-01]
 [-3.10071169e-01  3.92564590e-02 -9.49902522e-01]
 [-2.21066308e-01  2.07235365e-07 -9.75258780e-01]
 [-2.65577574e-01 -1.84601949e-01 -9.46250851e-01]
 [-7.91944115e-01 -2.92017206e-02 -6.09894891e-01]
 [-8.84912237e-02 -9.89400811e-01  1.15131831e-01]
 [ 6.28492508e-01 -6.12988948e-01 -4.78791935e-01]
 [ 7.28260110e-01 -4.73518839e-01 -4.95395924e-01]
 [-5.07368635e-03 -9.99572767e-01 -2.87844085e-02]
 [ 3.49295119e-01  1.16948013e-02 -9.36939780e-01]]


In [21]:
cropped_path='cropped.json'

In [24]:
print("Crop the original point cloud using a different polygon volume")
vol= o3d.visualization.read_selection_polygon_volume(cropped_path)

Crop the original point cloud using a different polygon volume


In [25]:
chair = vol.crop_point_cloud(pcd)
o3d.visualization.draw_geometries([chair],zoom=0.3412, front = [0.4257, -0.2125, -0.8795],
                                 lookat=[2.6172, 2.0475, 1.532],
                                 up=[-0.0694, -0.9768, 0.2024])

'read_selection_polygon_volume' read a json file that specifies polygon selection area
'vol.crop_point_cloud' crops the vol part from the original point cloud

In [41]:
def geometry(vis):
    o3d.visualization.draw_geometries(vis,zoom=0.3412, front = [0.4257, -0.2125, -0.8795],
                                 lookat=[2.6172, 2.0475, 1.532],
                                 up=[-0.0694, -0.9768, 0.2024])

In [33]:
#colors should range from 0-1
chair.paint_uniform_color([143/255,35/255,35/255])
geometry([chair])

In [34]:
#compute point cloud distance
dists=pcd.compute_point_cloud_distance(chair)
dists=np.asarray(dists)

In [36]:
ind=np.where(dists>0.01)[0]
pcd_without_chair=pcd.select_by_index(ind)
geometry([pcd_without_chair])

# Bounding volumes
open3d implements an 'AxisAlignedBoundingBox' and an 'OrientedBoundingBox' that can be used to crop the geometry

In [43]:
aabb=chair.get_axis_aligned_bounding_box()
aabb.color=(1,0,0)
obb=chair.get_oriented_bounding_box()
obb.color=(0,1,0)
geometry([chair,aabb,obb])

# Plane Segmentation
Open3D also supports segmententation of geometric primitives from point clouds using RANSAC. To find the plane with the largest support in the point cloud, we can use segment_plane. The method has three arguments: distance_threshold defines the maximum distance a point can have to an estimated plane to be considered an inlier, ransac_n defines the number of points that are randomly sampled to estimate a plane, and num_iterations defines how often a random plane is sampled and verified. The function then returns the plane as (a,b,c,d)
 such that for each point (x,y,z)
 on the plane we have ax+by+cz+d=0
. The function further returns a list of indices of the inlier points.

In [46]:
plane_model, inliers = pcd.segment_plane(distance_threshold=0.01,
                                        ransac_n=3,
                                        num_iterations=1000)
[a,b,c,d]=plane_model
print(f"Plane equation: {a:.2f}x + {b:.2f}y + {c:.2f}z + {d:.2f}=0")

inlier_cloud= pcd.select_by_index(inliers)
inlier_cloud.paint_uniform_color([1,0,0])
outlier_cloud = pcd.select_by_index(inliers, invert=True)
geometry([inlier_cloud, outlier_cloud])

Plane equation: -0.00x + 1.00y + 0.03z + -2.43=0
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 


# Surface Reconstruction

In [80]:
bunny = o3d.data.BunnyMesh()
mesh  = o3d.io.read_triangle_mesh(bunny.path)

[Open3D INFO] Downloading https://github.com/isl-org/open3d_downloads/releases/download/20220201-data/BunnyMesh.ply
[Open3D INFO] Downloaded to C:\Users\vedant/open3d_data/download/BunnyMesh/BunnyMesh.ply


In many scenarios we want to generate a dense 3D geometry, i.e., a triangle mesh. However, from a multi-view stereo method, or a depth sensor we only obtain an unstructured point cloud. To get a triangle mesh from this unstructured input we need to perform surface reconstruction. In the literature there exists a couple of methods and Open3D currently implements the following:

Alpha shapes [Edelsbrunner1983]

Ball pivoting [Bernardini1999]

Poisson surface reconstruction [Kazhdan2006]
The alpha shape [Edelsbrunner1983] is a generalization of a convex hull
Open3D implements the method create_from_point_cloud_alpha_shape that involves the tradeoff parameter alpha.

In [81]:
pcd = mesh.sample_points_poisson_disk(750)
o3d.visualization.draw_geometries([pcd])
alpha = 0.03
print(f"alpha={alpha:.3f}")
mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_alpha_shape(pcd, alpha)
mesh.compute_vertex_normals()
o3d.visualization.draw_geometries([mesh], mesh_show_back_face=True)

alpha=0.030


In [82]:
tetra_mesh, pt_map = o3d.geometry.TetraMesh.create_from_point_cloud(pcd)
for alpha in np.logspace(np.log10(0.5), np.log10(0.01), num=4):
    print(f"alpha={alpha:.3f}")
    mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_alpha_shape(
        pcd, alpha, tetra_mesh, pt_map)
    mesh.compute_vertex_normals()
    o3d.visualization.draw_geometries([mesh], mesh_show_back_face=True)

alpha=0.500
alpha=0.136
alpha=0.037
alpha=0.010


# Ball pivoting
The ball pivoting algorithm (BPA) [Bernardini1999] is a surface reconstruction method which is related to alpha shapes. Intuitively, think of a 3D ball with a given radius that we drop on the point cloud. If it hits any 3 points (and it does not fall through those 3 points) it creates a triangles. Then, the algorithm starts pivoting from the edges of the existing triangles and every time it hits 3 points where the ball does not fall through we create another triangle.

Open3D implements this method in create_from_point_cloud_ball_pivoting. The method accepts a list of radii as parameter that corresponds to the radii of the individual balls that are pivoted on the point cloud.

In [83]:
gt_mesh=mesh
gt_mesh.compute_vertex_normals()
pcd = gt_mesh.sample_points_poisson_disk(3000)
o3d.visualization.draw_geometries([pcd])

In [84]:
radii = [0.005, 0.01, 0.02, 0.04]
rec_mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_ball_pivoting(
    pcd, o3d.utility.DoubleVector(radii))
o3d.visualization.draw_geometries([pcd, rec_mesh])

# Poisson surface reconstruction
The Poisson surface reconstruction method [Kazhdan2006] solves a regularized optimization problem to obtain a smooth surface. For this reason, Poisson surface reconstruction can be preferable to the methods mentioned above, as they produce non-smooth results since the points of the PointCloud are also the vertices of the resulting triangle mesh without any modifications.

Open3D implements the method create_from_point_cloud_poisson which is basically a wrapper of the code of Kazhdan. An important parameter of the function is depth that defines the depth of the octree used for the surface reconstruction and hence implies the resolution of the resulting triangle mesh. A higher depth value means a mesh with more details.

In [85]:
dataset = o3d.data.EaglePointCloud()
pcd = o3d.io.read_point_cloud(dataset.path)
print(pcd)
o3d.visualization.draw_geometries([pcd],
                                  zoom=0.664,
                                  front=[-0.4761, -0.4698, -0.7434],
                                  lookat=[1.8900, 3.2596, 0.9284],
                                  up=[0.2304, -0.8825, 0.4101])

[Open3D INFO] Downloading https://github.com/isl-org/open3d_downloads/releases/download/20220201-data/EaglePointCloud.ply
[Open3D INFO] Downloaded to C:\Users\vedant/open3d_data/download/EaglePointCloud/EaglePointCloud.ply
PointCloud with 796825 points.
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 


In [86]:
print('run Poisson surface reconstruction')
with o3d.utility.VerbosityContextManager(
        o3d.utility.VerbosityLevel.Debug) as cm:
    mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(
        pcd, depth=9)
print(mesh)
o3d.visualization.draw_geometries([mesh],
                                  zoom=0.664,
                                  front=[-0.4761, -0.4698, -0.7434],
                                  lookat=[1.8900, 3.2596, 0.9284],
                                  up=[0.2304, -0.8825, 0.4101])

run Poisson surface reconstruction
[Open3D DEBUG] Input Points / Samples: 796825 / 368254
[Open3D DEBUG] #   Got kernel density: 0.122 (s), 190.516 (MB) / 190.516 (MB) / 1610 (MB)
[Open3D DEBUG] #     Got normal field: 0.43 (s), 297.707 (MB) / 297.707 (MB) / 1610 (MB)
[Open3D DEBUG] Point weight / Estimated Area: 2.623550e-06 / 2.090510e+00
[Open3D DEBUG] #       Finalized tree: 0.579 (s), 386.973 (MB) / 386.973 (MB) / 1610 (MB)
[Open3D DEBUG] #  Set FEM constraints: 0.799 (s), 337.559 (MB) / 386.973 (MB) / 1610 (MB)
[Open3D DEBUG] #Set point constraints: 0.233 (s), 317.273 (MB) / 386.973 (MB) / 1610 (MB)
[Open3D DEBUG] Leaf Nodes / Active Nodes / Ghost Nodes: 2945433 / 3365000 / 1209
[Open3D DEBUG] Memory Usage: 317.285 MB
[Open3D DEBUG] # Linear system solved: 1.875 (s), 390.105 (MB) / 390.105 (MB) / 1610 (MB)
[Open3D DEBUG] Got average: 0.046 (s), 305.879 (MB) / 390.105 (MB) / 1610 (MB)
[Open3D DEBUG] Iso-Value: 5.028477e-01 = 4.006817e+05 / 7.968250e+05
[Open3D DEBUG] #          To